In [40]:
#Libraries
import pandas as pd #For handling the dataset
from sklearn.neighbors import KNeighborsClassifier # The classification model I choosen
from sklearn.metrics import accuracy_score # To find out the accuracy of the model
from sklearn.feature_extraction.text import TfidfVectorizer # For converting the text into numeric why TfidfVectorizer bcoz it gives weight to unique words
from sklearn.model_selection import train_test_split # To split and train the model
from sklearn.multiclass import OneVsRestClassifier # To create multiple binary classification
import pickle # To create Pickle file(.pkl) inorder to deploy into the web application
import re # To clean the dataset
import string # To import the text into the clean_text function
import nltk 
from nltk.corpus import stopwords # To remove stopwords
from sklearn.preprocessing import LabelEncoder # For encoding the labelled data
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GODWIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
#Importing the dataset
dataset=pd.read_csv("D:\Projects\Resume Classifer\Resume Dataset.csv")
df=pd.DataFrame(dataset)
df["Resume"]

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\GODWIN\AppData\Local\Temp\ipykernel_20896\974931688.py:2: SyntaxWarning: invalid escape sequence '\P'
  dataset=pd.read_csv("D:\Projects\Resume Classifer\Resume Dataset.csv")


0      Skills * Programming Languages: Python (pandas...
1      Education Details \r\nMay 2013 to May 2017 B.E...
2      Areas of Interest Deep Learning, Control Syste...
3      Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4      Education Details \r\n MCA   YMCAUST,  Faridab...
                             ...                        
957    Computer Skills: â¢ Proficient in MS office (...
958    â Willingness to accept the challenges. â ...
959    PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...
960    COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...
961    Skill Set OS Windows XP/7/8/8.1/10 Database MY...
Name: Resume, Length: 962, dtype: object

In [42]:
#Going to clean the dataset 
#LowerCase - Done
df["Resume"] = df["Resume"].str.lower()
df.columns

Index(['Category', 'Resume'], dtype='object')

In [43]:
# #To remove stopwords 
#Stopword Removal - Done
stop_words=set(stopwords.words('english'))
df['Stopwordremoved_Resume'] = df['Resume'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['Stopwordremoved_Resume'] 



0      skills * programming languages: python (pandas...
1      education details may 2013 may 2017 b.e uit-rg...
2      areas interest deep learning, control system d...
3      skills â¢ r â¢ python â¢ sap hana â¢ table...
4      education details mca ymcaust, faridabad, hary...
                             ...                        
957    computer skills: â¢ proficient ms office (wor...
958    â willingness accept challenges. â positiv...
959    personal skills â¢ quick learner, â¢ eagerne...
960    computer skills & software knowledge ms-power ...
961    skill set os windows xp/7/8/8.1/10 database my...
Name: Stopwordremoved_Resume, Length: 962, dtype: object

In [44]:
# Removing the 

# 1.Hashtags
# 2.Mentions
# 3.special letters
# 4.URLs
# 5.Punctuations



def clean_text(text):
    text=re.sub(r'#\S+','',text)
    text=re.sub(r'@|S+','',text)
    text=re.sub(r'[^a-zA-Z0-9\s]','',text)
    text=re.sub(r'http\S+|www\S+','',text)
    text=text.translate(str.maketrans('','',string.punctuation))
    return text

df['Cleaned_Resume']=df['Stopwordremoved_Resume'].apply(clean_text)
df

,Category,Resume,Stopwordremoved_Resume,Cleaned_Resume
0,Data Science,skills * programming languages: python (pandas...,skills * programming languages: python (pandas...,skills programming languages python pandas nu...
1,Data Science,education details \r\nmay 2013 to may 2017 b.e...,education details may 2013 may 2017 b.e uit-rg...,education details may 2013 may 2017 be uitrgpv...
2,Data Science,"areas of interest deep learning, control syste...","areas interest deep learning, control system d...",areas interest deep learning control system de...
3,Data Science,skills â¢ r â¢ python â¢ sap hana â¢ table...,skills â¢ r â¢ python â¢ sap hana â¢ table...,skills r python sap hana tableau sap hana...
4,Data Science,"education details \r\n mca ymcaust, faridab...","education details mca ymcaust, faridabad, hary...",education details mca ymcaust faridabad haryan...
...,...,...,...,...
957,Testing,computer skills: â¢ proficient in ms office (...,computer skills: â¢ proficient ms office (wor...,computer skills proficient ms office word bas...
958,Testing,â willingness to accept the challenges. â ...,â willingness accept challenges. â positiv...,willingness accept challenges positive think...
959,Testing,"personal skills â¢ quick learner, â¢ eagerne...","personal skills â¢ quick learner, â¢ eagerne...",personal skills quick learner eagerness lear...
960,Testing,computer skills & software knowledge ms-power ...,computer skills & software knowledge ms-power ...,computer skills software knowledge mspower po...


In [45]:
#Created a new Dataframe Containing  the column "Cleaned_Resume" and adding the column "Category" 
cleaned_resume_df=df[['Cleaned_Resume']].copy()
# cleaned_resume_df=df[['Category']].copy()
cleaned_resume_df["Category"]=df["Category"]
cleaned_resume_df


,Cleaned_Resume,Category
0,skills programming languages python pandas nu...,Data Science
1,education details may 2013 may 2017 be uitrgpv...,Data Science
2,areas interest deep learning control system de...,Data Science
3,skills r python sap hana tableau sap hana...,Data Science
4,education details mca ymcaust faridabad haryan...,Data Science
...,...,...
957,computer skills proficient ms office word bas...,Testing
958,willingness accept challenges positive think...,Testing
959,personal skills quick learner eagerness lear...,Testing
960,computer skills software knowledge mspower po...,Testing


In [46]:
#Encoding(We do encoding for label)
le=LabelEncoder()
le.fit(df['Category'])
df['Category']=le.transform(df['Category'])


In [47]:
#Vectorization (We do vectorization for features )

Tfidf= TfidfVectorizer()
Tfidf.fit(df['Cleaned_Resume'])
vect_Resume=Tfidf.transform(df['Cleaned_Resume'])



In [48]:
#Splitting data for training and testing
X_train,X_test,y_train,y_test=train_test_split(vect_Resume,df['Category'],test_size=0.2,random_state=42)


In [49]:
#Selecting the model and appling OneVsRestClassifier
KNN=OneVsRestClassifier(KNeighborsClassifier())
KNN.fit(X_train,y_train)
ypred=KNN.predict(X_test)
accuracy_score(y_test,ypred)

0.9844559585492227

In [50]:
#Creating Pickle File
with open('Tfidf.pkl', 'wb') as f:
    pickle.dump(Tfidf, f)
with open('KNN.pkl', 'wb') as f:
    pickle.dump(KNN, f)
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(le, f)
